# The relationship between the Human Development Index and the prevalance of mental health disorders

## Research question

The aim of this analysis is to investigate the relationship between country-level socioeconomic status and the prevalence of mental health disorders. Studies have suggested that mental health is worse is relatively highly developed countries [(Barbalat & Franck, 2020)](https://bmjopen.bmj.com/content/bmjopen/10/4/e035055.full.pdf). A country's level of human development can be measured using the humand development index (HDI).

The HDI is a metric designed to quantify a country's socieeconomic status by using key dimensions of human development. The index is calculated using four key metrics: (1) life expectancy at birth - to asses a long and healthy life, (2) expected years of schooling - to asses access to knowledge of the younger generation, (3) average years of schooling - to assess access to knowledge of the older generation and (4) gross national income (GNI) per capita - to asses the standard of living. Each of these metrics is normalized to an index value and aggregated into the HDI using the following formula [(2020 HDR Technical Notes)](https://hdr.undp.org/system/files/documents/technical-notes-calculating-human-development-indices.pdf):

$HDI = (I_{Health} * I_{Education} * I_{Income})^\frac{1}{3}$

The HDI dataset contains HDI values from 1990 to 2021 The 'Global Trends in Mental Health Disorder' file contains the prevalence expressed in percentages for schizophrenia, bipolar disorder, eating disorders, anxiety disorders, drug use disorders, depression and alcohol use disorders from 1990 to 2017. 

### Data sources

The [Human Development Index](https://ourworldindata.org/human-development-index) was retrieved from Our World in Data (Roser, 2014) and originally published by the United Nations Development Programme. 

The [Mental Health dataset](https://www.kaggle.com/datasets/thedevastator/uncover-global-trends-in-mental-health-disorder?resource=download) was retrieved from kaggle by the author Amit. 



### Loading libraries

In [ ]:
# Loading needed libraries
import yaml
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import json
import matplotlib.pyplot as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, Label
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw
import scipy
import scipy.stats as stats

from sklearn import linear_model

### Defining functions

In [ ]:
# Getting data location from configuration file
def get_config():
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config
    
def data_inspection(df):
    
    # Displaying number of columns and rows, datatypes, missing values
    print(f'Dataset contains {df.shape[0]} rows and {df.shape[1]} columns. \n')
    print(f'Datatypes: \n{df.dtypes} \n')
    print(f'Missing data per column: \n{df.isnull().sum()}')

    # Displaying number of countries and years included
    print(f'\nNumber of countries included: {len(df.Entity.unique())}')
    print(f'Number of years included: {len(df.Year.unique())}')
    
    # Displaying timespan of included years=
    print(f'Timespan: {df.Year.min()}-{df.Year.max()}')
   

def reformat_mh():
    # Drop index columns
    df = mental_disorders.drop('index', axis='columns')

    # Rename columns
    df = df.rename(columns = {'Schizophrenia (%)':'Schizophrenia',
    'Bipolar disorder (%)':'BPD', 
    'Eating disorders (%)':'ED',
    'Anxiety disorders (%)':'Anxiety',
    'Drug use disorders (%)':'Drugs',
    'Depression (%)':'Depression',
    'Alcohol use disorders (%)':'Alcohol'
    })

    # Drop missing values
    df = df.dropna()

    # Changing datatypes
    df['Year'] = df['Year'].astype(int)
    df['Schizophrenia'] = df['Schizophrenia'].astype(float)
    df['BPD'] = df['BPD'].astype(float)
    df['ED'] = df['ED'].astype(float)
    
    return df

def correct_countries(df, dict):
    for x in dict.items():
        df['Entity'] = df['Entity'].replace(x[0], x[1])

    return df


### Data loading

In [ ]:
config = get_config()

# Human development index data
d1 = (config['dataset1'])
hdi = pd.read_csv(d1)

# Mental health disorder data
d2 = (config['dataset2'])
mental_disorders = pd.read_csv(d2)

# Shapefile containing geographical country data needed for visualisation
fp = (config['dataset3'])
# Reading file using geopandas (only loading needed columns)
map_df = gpd.read_file(fp)[['ADMIN', 'ADM0_A3', 'geometry']]

### Data inspection 

#### Mental health disorders

In [ ]:
# Viewing dataset
mental_disorders.head()

In [ ]:
# Inspecting dataframe, datatypes, column and row counts and missing values
data_inspection(mental_disorders)

Observations: 

1. There is a lot of missing data. This needs to be filtered out and checked if remaining data is still sufficient and if remaining countries correspond to the HDI dataset. 

2. Several of columns need to be converted to float. 

3. Column names can be shortened for convenience.

4. The dataset includes an unexpectedly high number of years.This needs to be examined further

In [ ]:
# Checking time span 
print(mental_disorders['Year'].unique())

It currently contains years that go too far back (including prehistorical data).This can also factor into the high number of missing data. The dataset needs to be filtered to only contain data from the years 1990 to 2017.

#### Human development index

In [ ]:
# Viewing the dataframe
hdi.head()

In [ ]:
# Data inspection
data_inspection(hdi)

Observations: 

1. There is missing data in the 'Code' column. However, 'Entity' has no missing data. Since these column essentialy contain the same information, 'Entity' can be used to identity countries instead of the country code.

2. Datatypes are correct.

3. Contains less countries than the other dataset, but includes a longer timespan (from 1990 to 2021). This, however, will be filtered later on when merging the dataset to only include data up until 2017.


### Data cleaning

In [ ]:
# Removing ising data, renaming columns and converting datatypes
mh_clean = reformat_mh()

# Renaming country names to match other dataset
dict1 = {'Czech Republic':'Czechia','Macedonia':'North Macedonia'}
dict2 = {'Serbia':'Republic of Serbia',
    'Democratic Republic of Congo':'Democratic Republic of the Congo',
    'Bahamas':'The Bahamas',
    'Tanzania':'United Republic of Tanzania',
    'United States':'United States of America',  
    'Timor':'East Timor',
    "Cote d'Ivoire":'Ivory Coast',
    'Congo':'Republic of the Congo'}
mh_clean = correct_countries(mh_clean, dict1)
mh_clean = correct_countries(mh_clean, dict2)

In [ ]:
# Renaming country names to match other dataframes
hdi_clean = correct_countries(hdi, dict2)

# Renaming columns geographical data to match other datasets
map_df.columns = ['Entity', 'Code', 'geometry']

# Dropping information for antartica so it won't be displayed later
map_df = map_df.drop(map_df.index[159])

### Data Exploration

In [ ]:
import numpy as np
from bokeh.plotting import figure, show
pn.extension()

def plot_histogram(Variable, year):
    
    p = figure(width=500, height=400, toolbar_location=None,
           title="Distribution of the selected variable:")
    if Variable == "Human Development Index":
        df = hdi_clean

    else:
        df = mh_clean
        
    # Plotting per year
    df = df[df['Year'] == year]
        
    begin = min(df[Variable])
    end = max(df[Variable])
    bins = np.linspace(begin, end, 30)
    hist, edges = np.histogram(df[Variable], density=True, bins=bins)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
             fill_color="darkslateblue", fill_alpha = 0.7, line_color="white")
    p.xaxis.axis_label = "Distribution"
    p.yaxis.axis_label = "Count"

    return p

years = list(range(min(mh_hdi.Year),max(mh_hdi.Year)+1))
variables = ["Schizophrenia", "BPD", "ED", "Anxiety", "Drugs", "Depression", "Alcohol","Human Development Index"]
mh = pn.interact(plot_histogram, Variable = variables, year = years)
pn.Row(mh)

### Data wrangling

In [ ]:
# Merging HDI with mental disorders
mh_hdi = pd.merge(mh_clean, hdi, on=['Entity', 'Year', 'Code'])

# Merging geographical data
mh_hdi_geo  = map_df.merge(mh_hdi, on=['Entity', 'Code'])

Both data sets are normalized

### Statistical analysis

#### Simple linear regression

Checking for any linear relationship between the HDI and the prevalance of the mental health disorders

In [ ]:
def simple_linear_regression(year,variable):
    
    # Plotting per year
    df = mh_hdi[mh_hdi['Year'] == year]
    
    # Setting plot parameters
    p = figure(width=500, height=400)
    x = df["Human Development Index"]
    y = df[variable]
    
    # Scatter plot
    p.circle(x,y, size=7, color="darkslateblue", alpha=0.6)
    
    # Initializing linear model
    regression_model = linear_model.LinearRegression()

    # Train the model using the data
    regression_model.fit(X = pd.DataFrame(df["Human Development Index"]), 
                     y = df[variable])
    
    # Trained model y-intercept
    intercept = regression_model.intercept_

    # Trained model coefficients
    slope = regression_model.coef_[0]
    
    # Getting model score
    score = regression_model.score(X = pd.DataFrame(df["Human Development Index"]), 
                     y = df[variable])
    
    # Adding model score (R2) to plot
    a = max(df["Human Development Index"]) - 0.15
    b = min(df[variable])
    
    mytext = Label(x=a, y=b, text = f'R\N{SUPERSCRIPT TWO} = {round(score,3)}', 
                       text_color = "darkred", text_font_size = "10pt")
    
    # Getting linear prediction
    y_predicted = [slope*i + intercept  for i in x]
    p.line(x,y_predicted,color='crimson', alpha = 0.5, line_width = 2,
            legend_label = 'y = '+str(round(slope,3))+'x + '+str(round(intercept,3)))
    
    p.add_layout(mytext)
    p.xaxis.axis_label = "Human Development Index (HDI)"
    p.yaxis.axis_label = "Prevalance (%)"
    p.legend.background_fill_alpha = 0.6
    
    return p 

In [ ]:
years = list(range(min(mh_hdi.Year),max(mh_hdi.Year)+1))
variables = ["Schizophrenia", "BPD", "ED", "Anxiety", "Drugs", "Depression", "Alcohol"]

mh = pn.interact(simple_linear_regression, year=years, variable = variables)
pn.Row(mh)

There appears to be a positive correlation between HDI and all the variables except depression and alcohol disorders.

#### Spearman's correlation test

# Only 2017 for now
df_2017 = mh_hdi[mh_hdi['Year'] == 2017]
df_2017 = df_2017[["Entity", "Code","Year", "Anxiety"]]

#Merge dataframes gdf and df_2016.
merged = map_df.merge(df_2017, on = ['Entity','Code'])
#Read data to json.
year_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(year_json)
merged

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '1': '1%', '2':'2%', '3':'3%', '4':'4%', '5':'5%', '6':'6%','7':'7%', '8': '>8%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'Global Anxiety prevalence, 2017', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Anxiety', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure online in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### References
Barbalat, G., & Franck, N. (2020). Ecological study of the association between mental illness with human development, income inequalities and unemployment across OECD countries. BMJ open, 10(4), e035055.

Max Roser (2014) - "Human Development Index (HDI)". Published online at OurWorldInData.org. Retrieved from: 'https://ourworldindata.org/human-development-index' [Online Resource]*

### Data visualisation

In [ ]:
# Only 2017 for now
df_2017 = mh_hdi[mh_hdi['Year'] == 2017]
df_2017 = df_2017[["Entity", "Code","Year", "Anxiety"]]

#Merge dataframes gdf and df_2016.
merged = map_df.merge(df_2017, on = ['Entity','Code'])
#Read data to json.
year_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(year_json)
merged

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '1': '1%', '2':'2%', '3':'3%', '4':'4%', '5':'5%', '6':'6%','7':'7%', '8': '>8%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'Global Anxiety prevalence, 2017', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Anxiety', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure online in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

#### References
Barbalat, G., & Franck, N. (2020). Ecological study of the association between mental illness with human development, income inequalities and unemployment across OECD countries. BMJ open, 10(4), e035055.

Max Roser (2014) - "Human Development Index (HDI)". Published online at OurWorldInData.org. Retrieved from: 'https://ourworldindata.org/human-development-index' [Online Resource]*